In [2]:
import streamlit as st
import joblib
import numpy as np
import pandas as pd
import re

# Load the heart disease model
model = joblib.load("rf_heart_disease_model.pkl")
st.title("Heart Disease Prediction System")

st.header("Option 1: Fill out patient information manually")

# --- Manual Input Form ---
age = st.number_input("Age", min_value=0, max_value=120, value=50)
sex = st.selectbox("Gender", ["Female", "Male"])
fbs = st.selectbox("Fasting Blood Sugar > 120 mg/dl", [0, 1])
max_hr = st.number_input("Maximum Heart Rate Achieved", value=150)
exang = st.selectbox("Exercise Induced Angina", [0, 1])
oldpeak = st.number_input("ST Depression (Oldpeak)", value=0.1)
ca = st.selectbox("Number of Major Vessels Colored (0–4)", [0, 1, 2, 3, 4])

# Chest pain types
acp = st.selectbox("Asymptomatic Chest Pain", [0, 1])
nacp = st.selectbox("Non-Anginal Chest Pain", [0, 1])
ntcp = st.selectbox("Non-Typical Chest Pain", [0, 1])
tcp = st.selectbox("Typical Chest Pain", [0, 1])

# ECG
nrec = st.selectbox("Normal ECG", [0, 1])
warec = st.selectbox("Wave Abnormal ECG", [0, 1])
vrec = st.selectbox("Ventricular ECG", [0, 1])

# Slope
sups = st.selectbox("Slope Upsloping", [0, 1])
sf = st.selectbox("Slope Flat", [0, 1])
sds = st.selectbox("Slope Downsloping", [0, 1])

# Thal
ft = st.selectbox("Fixed Thal", [0, 1])
nt = st.selectbox("Normal Thal", [0, 1])
rt = st.selectbox("Reversible Thal", [0, 1])

# Final input
manual_input = [[
    age, int(sex == "Male"), fbs, max_hr, exang, oldpeak, ca,
    acp, nacp, ntcp, tcp,
    nrec, warec, vrec,
    sups, sf, sds,
    ft, nt, rt
]]

columns = [
    "age", "sex", "fbs", "max_hr", "exang", "oldpeak", "ca",
    "acp", "nacp", "ntcp", "tcp",
    "nrec", "warec", "vrec",
    "sups", "sf", "sds",
    "ft", "nt", "rt"
]

df_manual = pd.DataFrame(manual_input, columns=columns)

if st.button("Predict (Manual)"):
    prob = model.predict_proba(df_manual)[0][1] * 100
    if prob >= 40:
        st.error(f"You have a Heart Disease. Probability: {prob:.2f}%")
    else:
        st.success(f"You are safe from any kind of Heart Diseases. Probability: {prob:.2f}%")

# --- Agentic AI Input ---
st.markdown("---")
st.header("Option 2: Paste natural language prompt")

prompt = st.text_area("Enter description (e.g., 'I'm a 59-year-old female...')")

if st.button("Predict (Natural Language)"):
    try:
        import string

        # Regex matches
        age_match = re.search(r"(\d+)-year-old", prompt)
        hr_match = re.search(r"heart rate[^0-9]*([0-9]+)", prompt.lower())
        st_match = re.search(r"st depression[^0-9]*([\d.]+)", prompt.lower())
        ca_match = re.search(r"(?:vessels[^0-9]*|fluoroscopy.*?)([0-4])", prompt.lower())

        # Extracted features with fallback values
        age = int(age_match.group(1)) if age_match else 50
        sex = int("male" in prompt.lower())
        fbs = int("blood sugar" in prompt.lower() and not ("not more than" in prompt.lower() or "less than" in prompt.lower()))
        exang = int(not ("do not have exercise" in prompt.lower() or "no exercise" in prompt.lower()))
        max_hr = int(hr_match.group(1)) if hr_match else 150
        oldpeak_str = st_match.group(1).strip(string.punctuation) if st_match else "0.1"
        oldpeak = float(oldpeak_str)
        ca = int(ca_match.group(1)) if ca_match else 0

        # Chest pain
        acp = int("asymptomatic" in prompt.lower())
        nacp = int("non-anginal" in prompt.lower())
        ntcp = int("non-typical" in prompt.lower())
        tcp = int("typical chest pain" in prompt.lower())

        # ECG
        nrec = int("ecg is normal" in prompt.lower())
        warec = int("wave abnormal" in prompt.lower())
        vrec = int("ventricular" in prompt.lower())

        # Slope
        sups = int("upsloping" in prompt.lower())
        sf = int("flat" in prompt.lower())
        sds = int("downsloping" in prompt.lower())

        # Thal
        ft = int("fixed thal" in prompt.lower())
        nt = int("normal thal" in prompt.lower())
        rt = int("reversible thal" in prompt.lower())

        agent_input = [[
            age, sex, fbs, max_hr, exang, oldpeak, ca,
            acp, nacp, ntcp, tcp,
            nrec, warec, vrec,
            sups, sf, sds,
            ft, nt, rt
        ]]

        df_agent = pd.DataFrame(agent_input, columns=columns)

        prob = model.predict_proba(df_agent)[0][1] * 100
        if prob >= 40:
            st.error(f"[Agentic AI] You have a Heart Disease. Probability: {prob:.2f}%")
        else:
            st.success(f"[Agentic AI] You are safe from any kind of Heart Diseases. Probability: {prob:.2f}%")
    except Exception as e:
        st.warning(f"Could not parse input properly. Error: {e}")

ModuleNotFoundError: No module named 'streamlit'